In [1]:
import sqlite3
import pandas as pd

# Récupération des données 

In [2]:
# Connexion à la base de données
conn = sqlite3.connect('response.db')
cursor = conn.cursor()

In [3]:
# Création des
cursor.execute('''CREATE TABLE IF NOT EXISTS customers (curtomer_id INTEGER PRIMARY KEY, customer_name VARCHAR(50), age INTEGER, gender VARCHAR(10))''')
cursor.execute('''CREATE TABLE IF NOT EXISTS products (product_id INTEGER PRIMARY KEY, product_name VARCHAR(50), category VARCHAR(50), price INTEGER)''')
cursor.execute('''CREATE TABLE IF NOT EXISTS stores (store_id INTEGER PRIMARY KEY, store_name VARCHAR(50), location VARCHAR(50))''')
cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                    transaction_id INTEGER PRIMARY KEY, 
                    customer_id INTEGER, 
                    product_id INTEGER, 
                    store_id INTEGER, 
                    date DATE, 
                    amount FLOAT,
                    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
                    ON DELETE CASCADE
                    ON UPDATE CASCADE,
                    FOREIGN KEY(product_id) REFERENCES products(product_id)
                    ON DELETE CASCADE
                    ON UPDATE CASCADE,
                    FOREIGN KEY(store_id) REFERENCES stores(store_id)
                    ON DELETE CASCADE
                    ON UPDATE CASCADE
                    )''')

In [4]:
# Récupération des données
data_customers = pd.read_csv('data/customers.csv')
data_products = pd.read_csv('data/products.csv')
data_stores = pd.read_csv('data/stores.csv')
data_transactions = pd.read_csv('data/transactions.csv')

In [5]:
# Insertion des données
data_customers.to_sql('customers', conn, if_exists='replace', index=False)
data_products.to_sql('products', conn, if_exists='replace', index=False)
data_stores.to_sql('stores', conn, if_exists='replace', index=False)
data_transactions.to_sql('transactions', conn, if_exists='replace', index=False)

1000

# Exploration des tables et Pré-analyse

### 1 - Lister les noms des colonnes et le type de données pour chaque table.

In [6]:
def get_describe(table_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    rows = cursor.fetchall()
    
    print(f"Table {table_name}")
    for i in rows:
        print(i[1:3])

In [7]:
get_describe('customers')

Table customers
('customer_id', 'INTEGER')
('customer_name', 'TEXT')
('age', 'INTEGER')
('gender', 'TEXT')


In [8]:
get_describe('products')

Table products
('product_id', 'INTEGER')
('product_name', 'TEXT')
('category', 'TEXT')
('price', 'REAL')


In [9]:
get_describe('stores')

Table stores
('store_id', 'INTEGER')
('store_name', 'TEXT')
('location', 'TEXT')


In [10]:
get_describe('transactions')

Table transactions
('transaction_id', 'INTEGER')
('customer_id', 'INTEGER')
('product_id', 'INTEGER')
('store_id', 'INTEGER')
('date', 'TEXT')
('amount', 'REAL')


### 2 - Vérifier s'il y a des doublons dans la table transactions sur la colonne transaction_id.

In [11]:
rqt = '''
    SELECT 
        transaction_id, 
        COUNT(*) as count
    FROM 
        transactions
    GROUP BY 
        transaction_id
    HAVING 
        COUNT(*) > 1;
'''

cursor.execute(rqt)
result = cursor.fetchall()
print(result)

[]


### 3 - Identifier les clés primaires et les clés étrangères de chaque table.

Table customers :
    - PRIMARY KEY : customer_id

Table store : 
    - PRIMARY KEY : store_id

Table products : 
    - PRIMARY KEY : product_id 

Table transaction : 
    - PRIMARY KEY : transaction_id 
    - FOREIGN KEY : customer_id, product_id, store_id 

###  4 - Vérifier si la colonne date dans la table transactions est stockée dans un format date, sinon, la convertir.

In [12]:
rqt = "PRAGMA table_info(transactions)"

cursor.execute(rqt)
result = cursor.fetchall()
print(result[4])

(4, 'date', 'TEXT', 0, None, 0)


### 5 - Vérifier s'il existe des valeurs nulles dans les colonnes customer_id, product_id, store_id des transactions.

In [13]:
rqt = '''
    SELECT 
        customer_id, 
        product_id, 
        store_id 
    FROM 
        transactions 
    WHERE 
        customer_id IS NULL 
        OR product_id IS NULL 
        OR store_id IS NULL;
'''

cursor.execute(rqt)
result = cursor.fetchall()
print(result)

[]


# Requête de base 

### 6- Lister toutes les transactions effectuées dans le magasin New York

In [14]:
rqt = '''
    SELECT *
    FROM transactions
    LEFT JOIN stores ON transactions.store_id = stores.store_id
    WHERE stores.location = 'New York';
'''

result = pd.read_sql_query(rqt, conn)
result


,transaction_id,customer_id,product_id,store_id,date,amount,store_id,store_name,location
0,7,146,78,1,2023-01-07,48.99,1,Store 1,New York
1,12,500,81,1,2023-01-12,11.31,1,Store 1,New York
2,29,70,52,1,2023-01-29,290.12,1,Store 1,New York
3,51,400,62,1,2023-02-20,733.48,1,Store 1,New York
4,76,452,29,1,2023-03-17,277.07,1,Store 1,New York
...,...,...,...,...,...,...,...,...,...
396,937,104,88,20,2025-07-25,905.52,20,Store 20,New York
397,941,370,74,20,2025-07-29,308.36,20,Store 20,New York
398,960,497,2,20,2025-08-17,127.38,20,Store 20,New York
399,978,292,95,20,2025-09-04,218.93,20,Store 20,New York


### 7- Récupérer les noms des produits achetés par le customer_id 35.

In [15]:
rqt = '''
    SELECT transactions.customer_id,
            transactions.product_id, 
            products.product_name
    FROM transactions
        LEFT JOIN products ON transactions.product_id = products.product_id
    WHERE transactions.customer_id = '35';
'''

result = pd.read_sql_query(rqt, conn)
result

,customer_id,product_id,product_name
0,35,8,Product 8
1,35,96,Product 96


### 8- Lister les différents types de catégories de produits disponibles.

In [16]:
rqt = '''
    SELECT DISTINCT category
    FROM products;
'''

result = pd.read_sql_query(rqt, conn)
result

,category
0,Books
1,Electronics
2,Furniture
3,Clothing


### 9- Trouver le montant total des transactions pour le store_id 12.

In [17]:
rqt = '''
        SELECT store_id,
                SUM(amount) as total_amount
        FROM transactions
        WHERE store_id = 12;
'''

result = pd.read_sql_query(rqt, conn)
result

,store_id,total_amount
0,12,26796.29


### 10- Lister tous les clients de sexe féminin.

In [18]:
rqt = '''
    SELECT * 
    FROM customers
    WHERE gender = 'Female';
'''

result = pd.read_sql_query(rqt, conn)
result

,customer_id,customer_name,age,gender
0,2,Customer 2,40,Female
1,7,Customer 7,55,Female
2,9,Customer 9,51,Female
3,10,Customer 10,22,Female
4,11,Customer 11,69,Female
...,...,...,...,...
226,490,Customer 490,53,Female
227,494,Customer 494,35,Female
228,496,Customer 496,59,Female
229,497,Customer 497,50,Female


# Fonctions d’Agrégation et GROUP BY

### 11- Calculer la moyenne des montants des transactions par magasin.

In [19]:
rqt = ''' 
    SELECT store_id,
        AVG(amount) as average_amount
    FROM transactions
    GROUP BY store_id;
'''

result = pd.read_sql_query(rqt, conn)
result


,store_id,average_amount
0,1,532.130000
1,2,496.529592
2,3,481.485789
3,4,563.986364
4,5,614.618200
5,6,465.471190
6,7,503.759412
7,8,461.190196
8,9,465.133200
9,10,504.817115


### 12- Trouver le magasin qui a réalisé le chiffre d'affaires le plus élevé.

In [20]:
rqt = '''
    SELECT 
        store_id, 
        SUM(amount) as SOMME
    FROM transactions
    GROUP BY store_id
    ORDER BY SOMME DESC
    LIMIT 1;
'''

result = pd.read_sql_query(rqt, conn)
result

,store_id,SOMME
0,5,30730.91


### 13- Calculer le montant total dépensé par chaque client.

In [21]:
rqt = '''
    SELECT customer_id,
        SUM(amount) AS total_depense
    FROM transactions
    GROUP BY customer_id;
'''

result = pd.read_sql_query(rqt, conn)
result

,customer_id,total_depense
0,1,2528.13
1,2,2108.46
2,3,427.58
3,4,1447.50
4,5,574.77
...,...,...
434,496,2437.91
435,497,1821.58
436,498,1672.92
437,499,486.39


### 14- Trouver le produit le plus vendu en termes de quantité.

In [22]:
rqt = '''
    SELECT product_id, 
        COUNT(product_id) AS nb_vente
    FROM transactions 
    GROUP BY product_id 
    ORDER BY nb_vente DESC
    LIMIT 1;
'''

result = pd.read_sql_query(rqt, conn)
result

,product_id,nb_vente
0,45,20


### 15- Lister les catégories de produits et le montant total des ventes par catégorie.

In [23]:
rqt = '''
    SELECT products.category AS category,
        SUM(amount) AS sum_vente
    FROM products
    LEFT JOIN transactions ON transactions.product_id = products.product_id 
    GROUP BY category;
'''

result = pd.read_sql_query(rqt, conn)
result

,category,sum_vente
0,Books,151828.24
1,Clothing,108614.35
2,Electronics,125958.34
3,Furniture,127514.47


# Clauses HAVING vs WHERE

### 16- Lister les clients qui ont effectué plus de 5 transactions.

In [24]:
rqt = '''
        SELECT customer_id,
                COUNT(customer_id) AS nb_vente
        FROM transactions
        GROUP BY customer_id
        HAVING nb_vente > 5;
    '''

result = pd.read_sql_query(rqt, conn)
result

,customer_id,nb_vente
0,61,6
1,141,6
2,277,6


### 17- Trouver les produits qui ont généré plus de 1000 en ventes totales.

In [25]:
rqt = '''
    SELECT product_id, 
        SUM(amount) AS vente_total
    FROM transactions
    GROUP BY product_id
    HAVING vente_total > 1000
    ORDER BY vente_total;
'''

result = pd.read_sql_query(rqt, conn)
result

,product_id,vente_total
0,68,1442.03
1,56,1982.06
2,76,1989.27
3,78,2019.85
4,10,2154.66
...,...,...
95,35,9014.35
96,74,9037.33
97,48,9187.81
98,45,9438.43


### 18- Lister les magasins ayant une moyenne de transaction supérieure à 200.

In [26]:
rqt = '''
    SELECT store_id,
        AVG(amount) AS transaction_avg
    FROM transactions 
    GROUP BY store_id
    HAVING transaction_avg > 200;
'''

result = pd.read_sql_query(rqt, conn)
result

,store_id,transaction_avg
0,1,532.130000
1,2,496.529592
2,3,481.485789
3,4,563.986364
4,5,614.618200
5,6,465.471190
6,7,503.759412
7,8,461.190196
8,9,465.133200
9,10,504.817115


### 19- Récupérer les produits appartenant à la catégorie Books et ayant généré plus de 500 en ventes.

In [27]:
rqt = '''
    SELECT products.product_name,
        SUM(transactions.amount) AS somme
    FROM products
        LEFT JOIN transactions ON transactions.product_id = products.product_id
    WHERE products.category = 'Books'
    GROUP BY products.product_id
    HAVING somme > 500;
'''

result = pd.read_sql_query(rqt, conn)
result

,product_name,somme
0,Product 1,4334.66
1,Product 2,5735.32
2,Product 3,5909.48
3,Product 6,3147.49
4,Product 8,4192.44
5,Product 9,7749.04
6,Product 14,7558.51
7,Product 20,5097.30
8,Product 31,3538.25
9,Product 33,6547.09


### 20- Trouver les clients ayant dépensé plus de 1000 au total.

In [28]:
rqt = '''
        SELECT customer_id,
                SUM(amount) AS somme
        FROM transactions
        GROUP BY customer_id
        HAVING somme > 1000;
'''

result = pd.read_sql_query(rqt, conn)
result

,customer_id,somme
0,1,2528.13
1,2,2108.46
2,4,1447.50
3,6,1934.12
4,7,1303.84
...,...,...
217,494,2268.27
218,495,1814.12
219,496,2437.91
220,497,1821.58


# Fonctions de Manipulation de Chaînes et Dates

### 21- Extraire l’année des dates de transactions et compter le nombre de transactions par année.

In [29]:
rqt = '''
    SELECT strftime('%Y', date) AS year,
        COUNT(transaction_id) AS nb_transaction
    FROM transactions
    GROUP BY year;
'''

result = pd.read_sql_query(rqt, conn)
result

,year,nb_transaction
0,2023,365
1,2024,366
2,2025,269


### 22- Concaténer le nom du client avec son âge.

In [30]:
print(sqlite3.version)

2.6.0


In [83]:
# rqt = '''
#     SELECT customer_name || ' ' || age AS concat FROM customers;
# '''

rqt = ''' 
    SELECT 
        CONCAT(customer_name, ' ', age) AS concat
    FROM customers;
'''

result = pd.read_sql_query(rqt, conn)
result

,concat
0,Customer 1 25
1,Customer 2 40
2,Customer 3 30
3,Customer 4 33
4,Customer 5 43
...,...
495,Customer 496 59
496,Customer 497 50
497,Customer 498 51
498,Customer 499 35


### 23- Remplacer tous les espaces par des underscores dans les noms des produits.

In [32]:
rqt = '''
    SELECT *, 
        REPLACE(product_name, ' ', '_') AS product_bis
    FROM products;
'''

result = pd.read_sql_query(rqt, conn)
result

,product_id,product_name,category,price,product_bis
0,1,Product 1,Books,14.02,Product_1
1,2,Product 2,Books,291.83,Product_2
2,3,Product 3,Books,295.06,Product_3
3,4,Product 4,Electronics,228.15,Product_4
4,5,Product 5,Furniture,419.20,Product_5
...,...,...,...,...,...
95,96,Product 96,Furniture,112.24,Product_96
96,97,Product 97,Furniture,139.35,Product_97
97,98,Product 98,Books,412.89,Product_98
98,99,Product 99,Furniture,139.99,Product_99


### 24- Transformer en majuscules les noms des magasins.

In [33]:
rqt = '''
    SELECT *, UPPER(store_name) AS upper_name
    FROM stores;
'''

result = pd.read_sql_query(rqt, conn)
result

,store_id,store_name,location,upper_name
0,1,Store 1,New York,STORE 1
1,2,Store 2,Los Angeles,STORE 2
2,3,Store 3,New York,STORE 3
3,4,Store 4,New York,STORE 4
4,5,Store 5,Phoenix,STORE 5
5,6,Store 6,Los Angeles,STORE 6
6,7,Store 7,Los Angeles,STORE 7
7,8,Store 8,Houston,STORE 8
8,9,Store 9,New York,STORE 9
9,10,Store 10,New York,STORE 10


# JOIN

### 25- Joindre la table transactions avec stores et afficher le nom du magasin pour chaque transaction.

In [34]:
rqt = ''' 
    SELECT transactions.transaction_id, 
        stores.store_name
    FROM transactions
    LEFT JOIN stores ON stores.store_id = transactions.store_id;                
'''

result = pd.read_sql_query(rqt, conn)
result

,transaction_id,store_name
0,1,Store 12
1,2,Store 16
2,3,Store 20
3,4,Store 20
4,5,Store 6
...,...,...
995,996,Store 8
996,997,Store 4
997,998,Store 14
998,999,Store 8


### 26- Joindre les tables transactions et products pour obtenir le nom du produit acheté dans chaque transaction.

In [35]:
rqt = '''
    SELECT transactions.transaction_id,
        products.product_name
    FROM transactions
    LEFT JOIN products ON products.product_id = transactions.product_id;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,transaction_id,product_name
0,1,Product 8
1,2,Product 34
2,3,Product 50
3,4,Product 46
4,5,Product 89
...,...,...
995,996,Product 62
996,997,Product 9
997,998,Product 22
998,999,Product 85


### 27- Faire une jointure entre transactions, products, et stores pour afficher le produit acheté, le magasin et le montant.

In [36]:
rqt = '''
    SELECT products.product_name,
        stores.store_name,
        transactions.amount
    FROM transactions 
        LEFT JOIN products ON products.product_id = transactions.product_id
        LEFT JOIN stores ON stores.store_id = transactions.store_id;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,product_name,store_name,amount
0,Product 8,Store 12,379.09
1,Product 34,Store 16,18.48
2,Product 50,Store 20,783.51
3,Product 46,Store 20,333.82
4,Product 89,Store 6,329.74
...,...,...,...
995,Product 62,Store 8,268.09
996,Product 9,Store 4,829.86
997,Product 22,Store 14,603.12
998,Product 85,Store 8,882.43


### 28- Lister les transactions pour les clients de plus de 30 ans.

In [37]:
rqt = '''
    SELECT transactions.*
    FROM transactions
        LEFT JOIN customers ON customers.customer_id = transactions.customer_id
    WHERE customers.age > 30;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,transaction_id,customer_id,product_id,store_id,date,amount
0,418,2,17,20,2024-02-22,556.39
1,666,2,100,3,2024-10-27,669.64
2,999,2,85,8,2025-09-25,882.43
3,446,4,93,3,2024-03-21,171.69
4,568,4,99,4,2024-07-21,847.98
...,...,...,...,...,...,...
781,345,498,60,15,2023-12-11,463.53
782,958,498,98,4,2025-08-15,283.84
783,127,499,34,10,2023-05-07,486.39
784,12,500,81,1,2023-01-12,11.31


### 29- Lister tous les produits achetés par les clients de sexe masculin.

In [38]:
rqt = '''
    SELECT transactions.*
    FROM transactions
        LEFT JOIN customers ON customers.customer_id = transactions.customer_id
    WHERE customers.gender = "Male";
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,transaction_id,customer_id,product_id,store_id,date,amount
0,32,1,83,3,2023-02-01,377.15
1,761,1,88,1,2025-01-30,886.60
2,871,1,35,9,2025-05-20,649.32
3,965,1,89,5,2025-08-22,615.06
4,152,3,24,9,2023-06-01,284.59
...,...,...,...,...,...,...
545,370,495,33,6,2024-01-05,106.43
546,186,498,74,7,2023-07-05,925.55
547,345,498,60,15,2023-12-11,463.53
548,958,498,98,4,2025-08-15,283.84


# Sous-requêtes (Subqueries) et CTE

### 30- Trouver les clients qui ont effectué une transaction pour un montant supérieur à la moyenne des transactions.

In [39]:
rqt = '''
    SELECT *
    FROM transactions
    WHERE amount > (SELECT AVG(amount) FROM transactions);
'''
rqt = pd.read_sql_query(rqt, conn)
rqt

,transaction_id,customer_id,product_id,store_id,date,amount
0,3,334,50,20,2023-01-03,783.51
1,6,379,61,4,2023-01-06,811.79
2,8,42,95,15,2023-01-08,515.03
3,10,28,10,13,2023-01-10,826.28
4,11,453,74,12,2023-01-11,888.98
...,...,...,...,...,...,...
497,993,351,61,12,2025-09-19,934.32
498,994,189,99,9,2025-09-20,724.74
499,997,380,9,4,2025-09-23,829.86
500,998,46,22,14,2025-09-24,603.12


31- Lister les produits ayant un prix supérieur au prix moyen de leur catégorie.

In [40]:
rqt = '''
    SELECT 
        CASE 
            WHEN category = 'Books' AND price > (SELECT AVG(price) FROM products WHERE category = 'Books') THEN product_id
            WHEN category = 'Electronics' AND price > (SELECT AVG(price) FROM products WHERE category = 'Electronics') THEN product_id
            WHEN category = 'Furniture' AND price > (SELECT AVG(price) FROM products WHERE category = 'Furniture') THEN product_id
            WHEN category = 'Clothing' AND price > (SELECT AVG(price) FROM products WHERE category = 'Clothing') THEN product_id
            ELSE Null
        END AS product,
            price,
            category
    FROM products 
    WHERE product IS NOT NULL
    ORDER BY category, price;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,product,price,category
0,47,265.57,Books
1,65,280.82,Books
2,50,291.19,Books
3,2,291.83,Books
4,3,295.06,Books
5,14,309.15,Books
6,55,325.64,Books
7,42,361.28,Books
8,8,388.96,Books
9,20,400.85,Books


### 32- Utiliser un CTE pour calculer le montant total des transactions par client, puis filtrer pour trouver ceux ayant dépensé plus de 500.

In [41]:
rqt = '''
    WITH transactionClient AS (
        SELECT transactions.customer_id AS client,
            SUM(transactions.amount) AS somme
        FROM transactions
        GROUP BY transactions.customer_id
    )
    SELECT client, 
        somme
    FROM transactionClient 
    WHERE somme > 500;
'''

result = pd.read_sql_query(rqt, conn)
result

,client,somme
0,1,2528.13
1,2,2108.46
2,4,1447.50
3,5,574.77
4,6,1934.12
...,...,...
338,494,2268.27
339,495,1814.12
340,496,2437.91
341,497,1821.58


### 33- Lister les produits qui ont été achetés plus de 10 fois et appartenant à la catégorie Electronics.

In [45]:
rqt ='''
    WITH productSell AS (
        SELECT 
            product_id AS prod_id,
            COUNT(transaction_id) AS nb_transac
        FROM transactions
        GROUP BY prod_id
        HAVING COUNT(transaction_id) > 10
    )
    SELECT prod_id
    FROM productSell 
        LEFT JOIN products ON products.product_id = productSell.prod_id;
'''

result = pd.read_sql_query(rqt, conn)
result

,prod_id
0,3
1,7
2,9
3,14
4,16
5,18
6,24
7,27
8,28
9,29


# Fonctions Fenêtre (Window Functions)

### 34- Attribuer un rang aux transactions en fonction du montant dans chaque magasin.

In [46]:
rqt = '''
    SELECT *, 
        RANK() OVER(PARTITION BY store_id ORDER BY amount DESC) AS rank 
    FROM transactions;    
'''

result = pd.read_sql_query(rqt, conn)
result



,transaction_id,customer_id,product_id,store_id,date,amount,rank
0,722,440,50,1,2024-12-22,994.80,1
1,415,114,90,1,2024-02-19,975.16,2
2,667,496,45,1,2024-10-28,965.60,3
3,203,287,73,1,2023-07-22,936.11,4
4,340,392,49,1,2023-12-06,924.09,5
...,...,...,...,...,...,...,...
995,452,392,29,20,2024-03-27,95.91,52
996,584,61,74,20,2024-08-06,69.02,53
997,598,445,21,20,2024-08-20,49.71,54
998,715,350,32,20,2024-12-15,42.45,55


### 35- Calculer la somme cumulée du montant des transactions pour chaque client.

In [53]:
rqt = '''
    SELECT customer_id,
        SUM(amount) OVER(PARTITION BY customer_id) AS montant
    FROM transactions
    GROUP BY customer_id;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,customer_id,montant
0,1,377.15
1,2,556.39
2,3,284.59
3,4,171.69
4,5,248.07
...,...,...
434,496,315.73
435,497,821.82
436,498,925.55
437,499,486.39


### 36- Attribuer un numéro de rang aux magasins basé sur le chiffre d'affaires total

In [66]:
rqt = '''
    WITH datAmount AS (
        SELECT store_id,
            SUM(amount) AS montant
        FROM transactions
        GROUP BY store_id
    )
    SELECT *, RANK() OVER(ORDER BY montant DESC) as ROW
    FROM datAmount;
'''

rqt = pd.read_sql_query(rqt, conn)
rqt

,store_id,montant,ROW
0,5,30730.91,1
1,17,29533.06,2
2,19,27947.47,3
3,13,27881.63,4
4,20,27781.14,5
5,11,27649.98,6
6,3,27444.69,7
7,15,26822.47,8
8,12,26796.29,9
9,10,26250.49,10


### 37- Calculer le pourcentage de contribution de chaque transaction au total des ventes du magasin.

In [81]:
rqt = '''
    WITH totalStore AS (
        SELECT store_id, 
            amount,
            SUM(amount) OVER(PARTITION BY store_id) AS total_magasin
        FROM transactions
    )
    SELECT store_id,
        ROUND((amount / total_magasin) * 100, 2) || '%' AS percent
    FROM totalStore;
'''

result = pd.read_sql_query(rqt, conn)
result

,store_id,percent
0,1,0.21%
1,1,0.05%
2,1,1.27%
3,1,3.21%
4,1,1.21%
...,...,...
995,20,3.26%
996,20,1.11%
997,20,0.46%
998,20,0.79%
